In [1]:
import numpy as np
import matplotlib.pyplot as plt
import sys 
sys.path.insert(0, './MotorNet/')
import motornet as mn
import os
os.environ["CUDA_VISIBLE_DEVICES"]="-1"    
import tensorflow as tf
from utils import state_dict_pack, state_dict_unpack, GA
import torch
from Enviroments import SeqEnv, SeqEnvSimple
from Tasks import SingleReach
from time import time
from tqdm import tqdm

In [2]:
plant = mn.plants.ReluPointMass24()
print("Muscle Configuration:")
print(plant.get_muscle_cfg())

Muscle Configuration:
{'UpperRight': {'n_fixation_points': 2, 'fixation body': [0, 1], 'coordinates': [[2.0, 2.0], [0.0, 0.0]], 'max_isometric_force': 500}, 'UpperLeft': {'n_fixation_points': 2, 'fixation body': [0, 1], 'coordinates': [[-2.0, 2.0], [0.0, 0.0]], 'max_isometric_force': 500}, 'LowerRight': {'n_fixation_points': 2, 'fixation body': [0, 1], 'coordinates': [[2.0, -2.0], [0.0, 0.0]], 'max_isometric_force': 500}, 'LowerLeft': {'n_fixation_points': 2, 'fixation body': [0, 1], 'coordinates': [[-2.0, -2.0], [0.0, 0.0]], 'max_isometric_force': 500}}


2022-08-09 09:01:16.495445: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2022-08-09 09:01:16.495472: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: mehrdad-kubuntu
2022-08-09 09:01:16.495476: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: mehrdad-kubuntu
2022-08-09 09:01:16.495544: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 510.47.3
2022-08-09 09:01:16.495559: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 510.47.3
2022-08-09 09:01:16.495563: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:310] kernel version seems to match DSO: 510.47.3
2022-08-09 09:01:16.495820: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU in

In [3]:
class GRU_model(torch.nn.Module):

    def __init__(self, **kwargs):
        super(GRU_model, self).__init__()

        self.recurrent_layer = torch.nn.RNN(input_size=4, hidden_size=30, num_layers=1, bias=False, bidirectional=False, batch_first=True)
        self.linear1 = torch.nn.Linear(30, 4)
        self.activation = torch.nn.ReLU()

    def forward(self, x):
        x, h = self.recurrent_layer(x, torch.randn(1,1,30))
        x = self.linear1(x)
        x = self.activation(x)
        return x

with torch.no_grad():
    model = GRU_model()

print('The model:')
print(model)

The model:
GRU_model(
  (recurrent_layer): RNN(4, 30, bias=False, batch_first=True)
  (linear1): Linear(in_features=30, out_features=4, bias=True)
  (activation): ReLU()
)


In [8]:
# Test a model
def test_model(agent):
    done = False
    max_duration = 5
    task = SingleReach(type='fix')
    env = SeqEnvSimple(plant, task, motor_noise_sd = 0)
    vis = env.get_init_state()
    Reward = [] # list of state, action, reward
    duration = 0

    param = state_dict_pack(model.state_dict())
    new_state_dict = state_dict_unpack(model.state_dict(), param, agent['params'])
    model.load_state_dict(new_state_dict)

    with torch.no_grad():
        while True:
            vis_feedback = torch.Tensor(vis.reshape(1,1,4))
            action = model(vis_feedback)
            action = np.squeeze(action.numpy())
            reward, new_vis, done = env.step(action)
            vis = new_vis.copy()
            Reward.append(reward)
            duration += 1
            if (done != 0) or duration > max_duration:
                break
    return np.sum(Reward)

In [9]:
ga = GA(num_pop=50, cost_fun=test_model, mut_rate=0.01, tournament_size=0.2, num_par_core= 10)
param = state_dict_pack(model.state_dict())
ga.run_GA(num_generations=50, num_var = len(param['val_flatten']))
ga.plot_cost()

  0%|          | 0/50 [00:00<?, ?it/s]

tensor([[[2.1054, 0.3939, 1.8611, 3.3088]]])tensor([[[0.0000, 3.2468, 2.6192, 0.0000]]])tensor([[[0.0000, 0.0000, 0.0000, 0.4294]]])tensor([[[0.4243, 0.0000, 3.9835, 2.9281]]])tensor([[[1.5434, 1.9803, 0.1160, 2.5636]]])tensor([[[3.7396, 0.0000, 0.0000, 0.0000]]])





tensor([[[4.1690, 0.0000, 0.0000, 0.4800]]])tensor([[[0.0000, 0.0000, 2.6440, 0.0000]]])tensor([[[0.8329, 8.7251, 2.0529, 0.0000]]])tensor([[[4.7889, 2.7995, 0.0000, 1.8551]]])



tensor([[[6.1838, 3.1782, 3.3538, 5.5256]]])tensor([[[0.0000, 5.7186, 0.4796, 2.1735]]])

tensor([[[0.0000, 0.5711, 2.6358, 0.0000]]])
tensor([[[0.0000, 0.3832, 0.0000, 2.6537]]])
tensor([[[1.7909, 0.6398, 1.5457, 0.0363]]])tensor([[[2.0474, 0.9858, 1.9000, 0.4630]]])

tensor([[[0.0000, 0.0000, 1.9506, 0.0000]]])tensor([[[0.2138, 0.0000, 1.6457, 2.6547]]])tensor([[[0.0000, 0.0737, 0.1265, 0.0000]]])tensor([[[0.3942, 3.0698, 0.0000, 0.0000]]])



tensor([[[1.1393, 0.0000, 0.0000, 4.3552]]])
tensor([[[0.0000, 0.0000, 2.0248, 2.3663]]])
tensor([[[

  2%|▏         | 1/50 [00:01<00:57,  1.18s/it]

tensor([[[0., 0., 0., 0.]]])tensor([[[0., 0., 0., 0.]]])tensor([[[0., 0., 0., 0.]]])tensor([[[5.2010, 0.0000, 0.0000, 0.0000]]])



tensor([[[2.2493, 0.0000, 0.0000, 0.0000]]])tensor([[[2.8015, 0.0000, 0.0000, 0.0000]]])

tensor([[[0.0000, 1.5650, 0.0000, 2.8329]]])tensor([[[0.0000, 1.1638, 0.0000, 0.0000]]])tensor([[[0.0000, 0.5074, 0.0000, 0.0000]]])
tensor([[[1.2038, 0.0000, 0.2930, 2.3028]]])

tensor([[[0.0000, 0.2407, 0.0000, 1.7521]]])
tensor([[[0.0000, 0.0000, 1.3084, 0.0000]]])tensor([[[0.7546, 0.0000, 0.0000, 4.2403]]])

tensor([[[0.5045, 0.0000, 1.4339, 1.8255]]])

tensor([[[0.0000, 0.5004, 0.0000, 2.2960]]])
tensor([[[0.0000, 0.4531, 0.0000, 0.0000]]])
tensor([[[0.0000, 0.0000, 1.4714, 1.9456]]])
tensor([[[1.7640, 0.2500, 1.7892, 1.3392]]])tensor([[[0.0000, 2.7094, 0.0000, 1.0364]]])

tensor([[[0.0000, 0.7406, 1.1817, 3.1267]]])
tensor([[[0., 0., 0., 0.]]])tensor([[[0.0000, 0.1324, 1.2349, 1.3087]]])

tensor([[[0., 0., 0., 0.]]])
tensor([[[2.0909, 1.2692, 0.0000, 3.3326]]])


  4%|▍         | 2/50 [00:02<00:57,  1.20s/it]

...

tensor([[[0., 0., 0., 0.]]])tensor([[[0.0000, 0.0000, 0.0000, 1.4525]]])tensor([[[0.1241, 0.0760, 0.0000, 0.8514]]])tensor([[[0.0000, 0.0000, 1.4641, 0.5587]]])tensor([[[0.8328, 0.0000, 0.7781, 0.0000]]])tensor([[[0.5109, 0.0000, 2.3452, 0.0000]]])tensor([[[0., 0., 0., 0.]]])



tensor([[[0.0000, 0.1631, 0.0000, 2.0775]]])



tensor([[[0.6314, 0.0000, 0.0000, 0.0000]]])tensor([[[0.0000, 0.4054, 3.0410, 0.8393]]])tensor([[[0.0000, 0.0000, 4.2564, 3.2726]]])

tensor([[[0.0000, 1.0175, 0.0000, 1.1213]]])

tensor([[[0.0000, 0.5008, 0.5799, 0.0000]]])tensor([[[0.0000, 0.0000, 0.6279, 1.9810]]])tensor([[[0.0000, 0.0000, 1.4665, 0.0000]]])


tensor([[[0.9501, 0.4264, 0.3148, 2.2030]]])
tensor([[[0.0000, 0.0000, 0.2038, 4.2518]]])tensor([[[0.0000, 0.0000, 2.5126, 2.7314]]])

tensor([[[0.0000, 3.0248, 1.8514, 0.0000]]])tensor([[[0.0000, 1.5792, 0.0000, 0.0000]]])

tensor([[[0.0000, 0.0000, 0.6894, 0.0000]]])tensor([[[0.0000, 1.8719, 1.8722, 1.1500]]])

tensor([[[0.0000, 3.6328, 1.8439, 0.0459]]

 20%|██        | 10/50 [00:12<00:49,  1.23s/it]

tensor([[[0., 0., 0., 0.]]])tensor([[[3.3733, 0.0000, 0.0000, 0.0000]]])

tensor([[[0., 0., 0., 0.]]])
tensor([[[0., 0., 0., 0.]]])
tensor([[[0.4383, 0.0000, 2.6305, 0.0000]]])tensor([[[0.0000, 0.0000, 0.2293, 0.2615]]])tensor([[[0.0000, 0.8099, 0.5037, 0.0000]]])tensor([[[0.0000, 0.7572, 1.1538, 0.0000]]])tensor([[[0.0000, 0.3084, 4.3697, 3.3198]]])tensor([[[7.5557, 0.4125, 0.5201, 0.0000]]])





tensor([[[0.0000, 0.7069, 0.0000, 0.0000]]])
tensor([[[0.0000, 2.2785, 0.0000, 0.0000]]])
tensor([[[0.0000, 0.0000, 1.0601, 0.0000]]])
tensor([[[2.3590, 0.0000, 0.0000, 0.2433]]])tensor([[[0.8289, 0.1715, 0.0000, 0.0000]]])

tensor([[[0., 0., 0., 0.]]])
tensor([[[0.0000, 1.4569, 0.0000, 2.8766]]])
tensor([[[0.0000, 1.5480, 1.2153, 0.0000]]])tensor([[[0.0000, 0.9246, 0.0000, 2.5570]]])

tensor([[[0.0000, 0.0000, 0.5821, 2.3428]]])tensor([[[0.2348, 0.0000, 0.7473, 0.4332]]])

tensor([[[0.0000, 3.1286, 0.7041, 0.7321]]])
tensor([[[0.0000, 3.0904, 3.9777, 0.0000]]])
tensor([[[1.2697, 2.1930, 1.0

 20%|██        | 10/50 [00:13<00:52,  1.31s/it]


KeyboardInterrupt: 